<a href="https://colab.research.google.com/github/WoradeeKongthong/raining_tomorrow_classification/blob/master/03_Raining_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on Feature Engineering in part 02  
Outliers : I'll cap the outliers in X_train. And cap the outliers in X_test using the boundaries of X_train.  
Missing values : I'll impute the missing values in categorical features with 'most frequent' value,  
and impute the missing values in numerical features with median.

In [0]:
# libraries
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# **Data Set**

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/WoradeeKongthong/raining_tomorrow_classification/master/weatherAUS.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

In [0]:
# drop RISK_MM column (Recommendation from data description in Kaggle)
df.drop(['RISK_MM'], axis = 1, inplace = True)

# Extract Year, Month, Day from Date column
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# drop Date column
df.drop(['Date'], axis = 1, inplace = True)

# select year 2016-2017 to train the model
df = df[df['Year'] > 2015]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25974 entries, 2474 to 142192
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Location       25974 non-null  object 
 1   MinTemp        25814 non-null  float64
 2   MaxTemp        25881 non-null  float64
 3   Rainfall       25710 non-null  float64
 4   Evaporation    10704 non-null  float64
 5   Sunshine       8019 non-null   float64
 6   WindGustDir    24514 non-null  object 
 7   WindGustSpeed  24514 non-null  float64
 8   WindDir9am     24398 non-null  object 
 9   WindDir3pm     24746 non-null  object 
 10  WindSpeed9am   25841 non-null  float64
 11  WindSpeed3pm   24854 non-null  float64
 12  Humidity9am    25686 non-null  float64
 13  Humidity3pm    24218 non-null  float64
 14  Pressure9am    23294 non-null  float64
 15  Pressure3pm    23289 non-null  float64
 16  Cloud9am       14674 non-null  float64
 17  Cloud3pm       12665 non-null  float64
 18  Te

In [0]:
X = df.drop(['RainTomorrow'], axis=1)
y = df['RainTomorrow']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [8]:
# create temp_df to combine X_train and y_train and cap the outliers
temp_df = X_train
temp_df['RainTomorrow'] = y_train

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
temp_df

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,Year,Month,Day,RainTomorrow
44301,Wollongong,12.0,18.9,0.0,NaN,NaN,SW,35.0,SW,SSE,19.0,17.0,49.0,53.0,1022.9,1020.5,NaN,NaN,15.6,18.4,No,2017,4,28,No
138614,Darwin,23.1,33.7,9.8,6.6,6.4,NW,35.0,ESE,WNW,15.0,26.0,74.0,75.0,1009.4,1006.0,5.0,7.0,28.9,30.7,Yes,2016,2,12,No
65645,MelbourneAirport,10.2,30.7,0.0,6.6,11.4,SSE,31.0,N,SSE,13.0,17.0,71.0,30.0,1025.1,1022.4,0.0,0.0,16.2,29.4,No,2017,3,18,No
102441,Nuriootpa,11.9,28.9,0.0,8.2,11.9,SE,50.0,S,SSE,19.0,9.0,64.0,33.0,1013.5,1011.1,1.0,NaN,19.4,26.3,No,2016,1,3,No
5706,BadgerysCreek,4.7,25.6,1.4,NaN,NaN,WNW,50.0,NE,WNW,7.0,28.0,66.0,31.0,1014.5,1011.1,NaN,NaN,15.2,24.2,Yes,2016,10,18,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44182,Wollongong,20.2,31.7,0.0,NaN,NaN,SSW,48.0,NaN,NW,0.0,22.0,65.0,39.0,1003.4,1000.1,7.0,8.0,23.8,30.7,No,2016,12,30,No
126363,Walpole,5.8,14.3,12.4,NaN,NaN,NaN,NaN,W,WNW,22.0,15.0,78.0,87.0,1016.0,1016.2,NaN,NaN,10.8,11.9,Yes,2016,8,9,Yes
114415,PearceRAAF,9.9,18.0,2.4,NaN,8.6,SW,41.0,SSE,W,11.0,11.0,63.0,48.0,1024.2,1023.3,5.0,7.0,14.4,17.5,Yes,2016,9,8,No
75408,Portland,9.0,13.7,2.0,0.4,NaN,W,37.0,NNW,WNW,17.0,22.0,94.0,74.0,1031.3,1030.3,4.0,8.0,9.4,12.7,Yes,2016,7,15,No


In [10]:
# cap the outliers in Training set

Q1 = temp_df.quantile(0.25)
Q3 = temp_df.quantile(0.75)
IQR = Q3 - Q1

lower_cap = Q1 - 1.5*IQR
upper_cap = Q3 + 1.5*IQR

features = lower_cap.index.values

for feature in features :
  temp_df[feature] = np.where(temp_df[feature]<lower_cap[feature],lower_cap[feature], temp_df[feature])
  temp_df[feature] = np.where(temp_df[feature]>upper_cap[feature],upper_cap[feature], temp_df[feature])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
# get y_train and X_train from temp_df
X_train = temp_df.drop(['RainTomorrow'], axis=1)
y_train = temp_df['RainTomorrow']

In [12]:
# cap outliers on the test set
for feature in features :
  X_test[feature] = np.where(X_test[feature]<lower_cap[feature],lower_cap[feature], X_test[feature])
  X_test[feature] = np.where(X_test[feature]>upper_cap[feature],upper_cap[feature], X_test[feature])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


# **Create Model Pipeline**

**Create Preprocessor : ColumnTransformer of numerical and categorical features**

In [0]:
numerical_features = [x for x in X.columns if df[x].dtype != 'object']

numeric_transformer = Pipeline(steps=[
          ('imputer', SimpleImputer(strategy='median')),
          ('scaler', MinMaxScaler())
])

categorical_features = [x for x in X.columns if df[x].dtype == 'object']

categorical_transformer = Pipeline(steps=[
          ('imputer', SimpleImputer(strategy='most_frequent')),
          ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
          ('num', numeric_transformer, numerical_features),
          ('cat', categorical_transformer, categorical_features)
    ]
)

**Create model**

In [0]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2, n_jobs = -1)

**Create Pipeline**

In [0]:
clf = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('model', model)
])

# **Cross Validation on Training set**

In [16]:
accuracy = cross_val_score(clf,X_train,y_train,cv=10)
print('accuracy : ', accuracy)
print('mean : ', accuracy.mean())
print('std : ', accuracy.std())

accuracy :  [0.78200192 0.80028874 0.78873917 0.79162656 0.79643888 0.7786333
 0.79018287 0.79307026 0.79836381 0.78189697]
mean :  0.7901242491321838
std :  0.006997441411324103


Note : low bias and very low variance

# **Training and Test sets**

In [17]:
# Make sure your X_test is capped before running this cell

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print('\n\nTraining and Test Sets result')
print('\naccuracy score : ', accuracy_score(y_test,y_pred))
print('\nconfusion matrix : \n', confusion_matrix(y_test, y_pred))
print('\nclassification report : \n', classification_report(y_test,y_pred))

print('Training set score : ',clf.score(X_train,y_train))
print('Test set score : ',clf.score(X_test,y_test))



Training and Test Sets result

accuracy score :  0.7763233878729547

confusion matrix : 
 [[3642  311]
 [ 851  391]]

classification report : 
               precision    recall  f1-score   support

          No       0.81      0.92      0.86      3953
         Yes       0.56      0.31      0.40      1242

    accuracy                           0.78      5195
   macro avg       0.68      0.62      0.63      5195
weighted avg       0.75      0.78      0.75      5195

Training set score :  0.8470089994706194
Test set score :  0.7763233878729547


Note : there is overfitting in the model

Try adjust number of neighbors 

In [19]:
numerical_features = [x for x in X.columns if df[x].dtype != 'object']

numeric_transformer = Pipeline(steps=[
          ('imputer', SimpleImputer(strategy='median')),
          ('scaler', MinMaxScaler())
])

categorical_features = [x for x in X.columns if df[x].dtype == 'object']

categorical_transformer = Pipeline(steps=[
          ('imputer', SimpleImputer(strategy='most_frequent')),
          ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
          ('num', numeric_transformer, numerical_features),
          ('cat', categorical_transformer, categorical_features)
    ]
)

for k in range(5,15):
  model = KNeighborsClassifier(n_neighbors = k, metric = 'minkowski', p = 2, n_jobs = -1)

  clf = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('model', model)
  ])
  
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  print('KNN model : n_neighbors = {}'.format(k))
  print('Training set score : ',clf.score(X_train,y_train))
  print('Test set score : ',clf.score(X_test,y_test))


KNN model : n_neighbors = 5
Training set score :  0.8470089994706194
Test set score :  0.7763233878729547
KNN model : n_neighbors = 6
Training set score :  0.8335338562972232
Test set score :  0.7807507218479307
KNN model : n_neighbors = 7
Training set score :  0.8378170268059099
Test set score :  0.782483156881617
KNN model : n_neighbors = 8
Training set score :  0.8289619327205352
Test set score :  0.785563041385948
KNN model : n_neighbors = 9
Training set score :  0.8320900909572164
Test set score :  0.7890279114533205
KNN model : n_neighbors = 10
Training set score :  0.8262187785745223
Test set score :  0.7884504331087584
KNN model : n_neighbors = 11
Training set score :  0.8273256653351941
Test set score :  0.7934552454282965
KNN model : n_neighbors = 12
Training set score :  0.822994369315174
Test set score :  0.7897978825794033
KNN model : n_neighbors = 13
Training set score :  0.8265556571538573
Test set score :  0.7926852743022137
KNN model : n_neighbors = 14
Training set sco